Going to start a ground  u p rework fothe data based on features suggested by our favourite co-pilot. It seems like we can get a decent answer based on a lightweight model, by processing the audio files using librosa. The features we're looking to extract are: Mel-Frequency Spectral Coefficients(MFCC) 
Spectral Centroid
Spectral Bandwidth
Spectral Flatness
Spectral Contrast
Time Domain features - Zero Crossing rate (ZCR)
Root Mean Square Energy (RMSE)
Temporal features - Short-time energy
Tempo

Tips for Efficient Processing on Raspberry Pi 5

    Downsampling:
        Use a lower sampling rate (e.g., 16 kHz) unless high-frequency information is crucial.
    Windowing:
        Apply short-time Fourier transform (STFT) with small windows (e.g., 20–50 ms) for manageable computational loads.
    Batch Processing:
        Process audio in chunks to avoid memory and CPU spikes.
    Feature Dimensionality Reduction:
        Reduce the number of features using techniques like Principal Component Analysis (PCA) after extraction.
    Quantized Models:
        Use a lightweight, quantized ML model optimized for edge devices.

Import basic pacages for data processing . We're going to use Librosa to process the data as it seems to be  more lightweight than alternatives.   

In [5]:
import librosa
import numpy as np
from numpy import nan
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import math

Preprocessing of audio, we are resampling, normalising and trimming silence.

Resampling will normalise the sampling rate, reducing one variable between datasets. I assume this means that we will want to resamle audio in the input stream.

Normalise standardises the audio amplitude, apparently mimising the influence of recording volume, to prevent outlier high or low sounds from affecting the  model training. 

Trimming silence to remove useless pieces of data to save on computation, and reduce the affect silence could have on affecting our model weights. (Need to check this, feel like it should be heavily caveated.) I have now checked this, it makes sense that silence being a recurring feature across classes could lead to issues, but having a separate class specifically for silence callsed (no event) would be good potentially.

Going to start with a single wav file as a test.

In [ ]:
TestAudioPath = "audio/fold1/7061-6-0-0.wav"

def extract_features(file_path, sr=16000, n_mfcc=13):
    # Load the audio file
    audio, sample_rate = librosa.load(TestAudioPath, sr=sr)

    # Extract the MFCC features
    mfccs = librosa.features.mfcc(y=audio, sr=sample_rate, n_mfcc = n_mfcc)

    # Compute the mean of the extracted features across  frames
    mfccs_mean = np.mean(mfccs, axis=1)

    #TODO: This still needs to be finished

I've questioned chatgpt on why it wants  to normalise the MFCCs to a human frequency (using the mel scale.) It has agreed this doesnt really make sense. I think I will try to go down the path of doing both mel scale and none mel scale feature sets, build a model from both and see how they compare. Potentially may be worth stacking them? I don't see how applying a log scale is going to help much though...

In [ ]:
def extract_spectral_features(audio, sr = 16000):
    
    audio, sample_rate = librosa.load(audio, sr=sr)
    print("Audio data type:", type(audio))
    #  Short-Time Fourier Transform (STFT)
    # Audio is the inputted audio waveform, it transforms it into the time frequency domain.
    # we use abs becase it takes the magnitude, or the "strength" of each freuency
    # np.abs is a numpy feature to take an array and convert all values to positives.
    stft = np.abs(librosa.stft(audio))

    
    # Spectral Features, the spectral centroied is seen as the "brightness" of a sound/ Gunshots typically? Have higher spectral centroids due to high energy  at high frequencies.
    centroid = np.mean(librosa.feature.spectral_centroid(S=stft, sr=sr))
    # the bandwidth shows the width of the distributionn of frequencies, according to ChatGPT gunshots will have a higher than typical sound badnnwidth
    bandwidth = np.mean(librosa.feature.spectral_bandwidth(S=stft, sr=sr))
    # Apparently Gunshots are also generally high spectral flatness.
    flatness = np.mean(librosa.feature.spectral_flatness(S=stft))
    
    # Temporal Features. The onsert_env seems like a really important one, it's the sharpness/prominence of the onset of the sound.
    # Due to what a gunshot is, I expect this to be MASSIVe compared to a usual sound.
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
    # Captures the maximum intensity of the sound, which will probably  be higher than an average sound.
    peak_amplitude = np.max(np.abs(audio))
    
    # Combine features into a single array representing the audio clip
    features = [centroid, bandwidth, flatness, onset_env, peak_amplitude]
    return features

I'm going to try the feature set as described above. But want to bare in mind Mariams paper. The authors in [1] stress the fact that the orders in which these features are stacked is significantly impactful on the final result. Therefore, following their recommended order, the features are horizontally stacked in the following order: spectral contrast, tonnetz, chromagram, Mel-spectrogram, and MFCC. The resulting vectors are then inputted into the classifier.

Will try and test both.

In [21]:
all_features = []  # A list to hold features for each audio clip

#Single test audio file for now, need to  do some processing of files first.
TestAudioPath = "audio/fold1/7061-6-0-0.wav"

# For each audio clip, append its features
features = extract_spectral_features(TestAudioPath, sr=16000)
all_features.append(features)

# Create a DataFrame
df = pd.DataFrame(all_features, columns=["Centroid", "Bandwidth", "Flatness","Onset Env", "PeakAmplitude"])

# Save to CSV
df.to_csv("spectral_features.csv", index=False)

# Check the extracted features
df.head()

Audio data type: <class 'numpy.ndarray'>


,Centroid,Bandwidth,Flatness,Onset Env,PeakAmplitude
0,3115.782355,1975.385019,0.17813,"[0.0, 0.0, 0.0, 6.9627, 0.24204145, 0.15000804...",0.833382


I want to try and run a sample of different sounds and see what the different types of sound look like visually. First need to do some data loading of the metadata

In [25]:
# Load the metadata:
metadata = pd.read_csv('./data-description.csv')

Each row in the dataframe named `metadata' represents the metadata for one audio file. For each audio file, the following details are included:
- Name of the file.
- The recording sound ID.
- The start time of the slice in the original audio.
- The end time of the slice in the original audio.
- salience, an indicator for whether the sound slice is a foreground sound (1) or background sound (2).
- ID of the class to which the sound slice belongs.
- Name of the class to which the sound slice belongs.

In [27]:
# Add column calculating the length of the clips
metadata['length'] = metadata['end'] - metadata['start']


,slice_file_name,fsID,start,end,salience,fold,classID,class,length
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark,0.317551
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing,4.000000
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing,4.000000
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing,4.000000
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing,4.000000


In [24]:
# Label map of the different sound classes:
label_map = {
    'air_conditioner': 0,
    'car_horn': 1,
    'children_playing': 2,
    'dog_bark': 3,
    'drilling': 4,
    'engine_idling': 5,
    'gun_shot': 6,
    'jackhammer': 7,
    'siren': 8,
    'street_music': 9,
}

def get_class_name(idx):
    return list(label_map.keys())[list(label_map.values()).index(idx)]

In [31]:
# Get all the folders 
# Array of paths to each audio folder
AUDIO_DIR = './audio' # path to audio files directory
folders = ['fold{}'.format(x) for x in range(1, 12)] 



In [34]:
for fold in folders:
    print('collecting {}...'.format(fold), end = "")
    files = librosa.util.find_files('{}/{}'.format(AUDIO_DIR, fold), ext=['wav'])
    files = np.asarray(files)
    for file in files:
        if '.wav' in file:
            name = file.split('/').pop()
            idx = list(df.index[df['slice_file_name'] == name])[0]
    print("done!")


collecting fold1...done!
collecting fold2...done!
collecting fold3...done!
collecting fold4...done!
collecting fold5...done!
collecting fold6...done!
collecting fold7...done!
collecting fold8...done!
collecting fold9...done!
collecting fold10...done!
collecting fold11...done!


In [36]:
import os

def rename_wav_files(directory):
    """
    Renames .wav files in the specified directory by chopping the first 4 digits,
    the last 2 digits, and appending an incremental counter.
    
    Args:
        directory (str): Path to the directory containing .wav files.
    """
    counter = 0  # Initialize the counter
    
    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".wav"):  # Process only .wav files
            # Chop the filename as specified
            parts = filename.split("-")
            if len(parts) >= 4:
                new_name = f"{counter}-{parts[1]}.wav"
                old_path = os.path.join(directory, filename)
                new_path = os.path.join(directory, new_name)
                
                # Rename the file
                os.rename(old_path, new_path)
                print(f"Renamed: {filename} -> {new_name}")
                
                # Increment the counter
                counter += 1

# Specify the directory containing your .wav files
directory = "audio/fold1"  # Replace with the actual path
rename_wav_files(directory)


Renamed: 146186-5-0-5.wav -> 0-5.wav
Renamed: 65749-3-1-12.wav -> 1-3.wav
Renamed: 157867-8-0-0.wav -> 2-8.wav
Renamed: 113205-5-1-6.wav -> 3-5.wav
Renamed: 157867-8-0-21.wav -> 4-8.wav
Renamed: 164312-3-3-0.wav -> 5-3.wav
Renamed: 134717-0-0-0.wav -> 6-0.wav
Renamed: 102305-6-0-0.wav -> 7-6.wav
Renamed: 103074-7-4-6.wav -> 8-7.wav
Renamed: 172314-9-0-51.wav -> 9-9.wav
Renamed: 177621-0-0-104.wav -> 10-0.wav
Renamed: 165067-2-0-35.wav -> 11-2.wav
Renamed: 176787-5-0-22.wav -> 12-5.wav
Renamed: 40722-8-0-1.wav -> 13-8.wav
Renamed: 146186-5-0-6.wav -> 14-5.wav
Renamed: 103258-5-0-4.wav -> 15-5.wav
Renamed: 69304-9-0-15.wav -> 16-9.wav
Renamed: 17592-5-0-0.wav -> 17-5.wav
Renamed: 132016-9-0-11.wav -> 18-9.wav
Renamed: 172314-9-0-40.wav -> 19-9.wav
Renamed: 40722-8-0-7.wav -> 20-8.wav
Renamed: 180937-7-1-10.wav -> 21-7.wav
Renamed: 180937-7-4-14.wav -> 22-7.wav
Renamed: 191431-9-0-61.wav -> 23-9.wav
Renamed: 24074-1-0-10.wav -> 24-1.wav
Renamed: 150341-3-1-0.wav -> 25-3.wav
Renamed: 17762